### Create my own model to test Scikit-Learn server
* The model need to be built using the same python package dependency that kubeflow ships a specific version of sklearnserver with. Without adherence to this constraint, the model deserialization during serving time would fail. the deployed inference service will not start successfully.
* See https://github.com/kubeflow/kfserving/blob/master/python/sklearnserver/setup.py

In [ ]:
from sklearn import svm
from sklearn import datasets
from joblib import dump
clf = svm.SVC(gamma='scale')
iris = datasets.load_iris()
X, y = iris.data, iris.target
clf.fit(X, y)
dump(clf, 'model.joblib')

### Copy the model to S3/Minio for serving input
* mc is a minio client that will need to be installed
```bash
$ mc config host list
<<-- output snipped -->>
k8s
  URL       : http://minio-service.kubeflow.svc.cluster.local:9000
  AccessKey : minio
  SecretKey : minio123
  API       : S3v4
  Lookup    : auto
<<-- output snipped -->>
$ mc mb k8s/kfserving-samples # create the toplevel bucket
$ mc cp model.joblib k8s/kfserving-samples/models/sklearn/iris/model.joblib # copy the model to S3
```

### Create yaml specifications for deploying to KFServing system

In [ ]:
namespace='pdas'
account_name='sa'
model_name="sklearn-iris"

s3_secret=f"""apiVersion: v1
kind: Secret
metadata:
  name: mysecret
  namespace: {namespace}
  annotations:
     serving.kubeflow.org/s3-endpoint: minio-service.kubeflow:9000 # replace with your s3 endpoint
     serving.kubeflow.org/s3-usehttps: "0" # by default 1, for testing with minio you need to set to 0
type: Opaque
data:
  awsAccessKeyID: bWluaW8=
  awsSecretAccessKey: bWluaW8xMjM=
"""

service_account=f"""apiVersion: v1
kind: ServiceAccount
metadata:
  name: {account_name}
  namespace: {namespace}
secrets:
  - name: mysecret
"""

sklearn_inference=f"""apiVersion: "serving.kubeflow.org/v1alpha2"
kind: "InferenceService"
metadata:
  name: "{model_name}"
  namespace: {namespace}
spec:
  default:
    predictor:
      serviceAccountName: {account_name}
      sklearn:
        runtimeVersion: "v0.3.0"
        storageUri: "s3://kfserving-samples/models/sklearn/iris"
"""

### Submit KFServing yaml specifications to k8s API server
* The programmatic actions are equivalent to following commands:
```bash
$ kubectl apply -f s3_secret.yaml
$ kubectl apply -f service_account.yaml
$ kubectl apply -f sklearn_inference.yaml
```

In [ ]:
print(s3_secret)
with open("s3_secret.yaml", "w") as f:
    f.write(s3_secret)

In [ ]:
print(service_account)
with open("service_account.yaml", "w") as f:
    f.write(service_account)

In [ ]:
print(sklearn_inference)
with open("sklearn_inference.yaml", "w") as f:
    f.write(sklearn_inference)

In [50]:
!kubectl apply -f s3_secret.yaml
!kubectl apply -f service_account.yaml
!kubectl apply -f sklearn_inference.yaml

secret/mysecret created
serviceaccount/sa created
inferenceservice.serving.kubeflow.org/sklearn-iris created


### Input data for testing Iris inference service

In [ ]:
import json
iris_input_json={
  "instances": [
    [6.8,  2.8,  4.8,  1.4],
    [6.0,  3.4,  4.5,  1.6]
  ]
}
print(iris_input_json)
with open("iris_input.json", "w") as f:
    json.dump(iris_input_json, f)

### Test the deployed model

In [62]:
inference_service_dns=f"{model_name}-predictor-default.{namespace}.svc.cluster.local"
inference_service_url=f"http://{inference_service_dns}/v1/models/{model_name}:predict"
curl_command=f'curl -H "Host: {inference_service_dns}" {inference_service_url} -d @./iris_input.json'
print(curl_command)
import subprocess
cp=subprocess.run(["curl", "-H", f"Host: {inference_service_dns}", inference_service_url,
                   "-d", "@./iris_input.json" ], stdout=subprocess.PIPE)
print(f"{cp.stdout}")

curl -H "Host: sklearn-iris-predictor-default.pdas.svc.cluster.local" http://sklearn-iris-predictor-default.pdas.svc.cluster.local/v1/models/sklearn-iris:predict -d @./iris_input.json
b'{"predictions": [1, 1]}'


### Clean up

In [ ]:
!kubectl delete -f sklearn_inference.yaml
!kubectl delete -f service_account.yaml
!kubectl delete -f s3_secret.yaml